In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv(r"C:\Users\HP\Downloads\diamonds (1).csv")
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.1+ MB


In [5]:
X = df.drop('price',axis=1)
y = df['price']
X.shape,y.shape

((53940, 9), (53940,))

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((40455, 9), (13485, 9), (40455,), (13485,))

In [7]:
from sklearn.preprocessing import MinMaxScaler,OrdinalEncoder
from sklearn.compose import ColumnTransformer
mms = MinMaxScaler()
cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_order = ['J', 'I', 'H', 'G', 'F', 'E', 'D']
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
oe = OrdinalEncoder(categories=[cut_order,color_order,clarity_order])

In [8]:
ct =ColumnTransformer([('MinMaxScaling',mms,['depth','table','x','y','z']),('OrdinalEncoding',oe,['cut','color','clarity'])],
                      remainder='passthrough',verbose=False,verbose_feature_names_out=False).set_output(transform='pandas')
ct

ColumnTransformer(remainder='passthrough',
                  transformers=[('MinMaxScaling', MinMaxScaler(),
                                 ['depth', 'table', 'x', 'y', 'z']),
                                ('OrdinalEncoding',
                                 OrdinalEncoder(categories=[['Fair', 'Good',
                                                             'Very Good',
                                                             'Premium',
                                                             'Ideal'],
                                                            ['J', 'I', 'H', 'G',
                                                             'F', 'E', 'D'],
                                                            ['I1', 'SI2', 'SI1',
                                                             'VS2', 'VS1',
                                                             'VVS2', 'VVS1',
                                                             'IF']]),
                                 ['cut', 'color', 'clarity'])],
                  verbose_feature_names_out=False)

In [9]:
X_train_transformed = ct.fit_transform(X_train)
X_train_transformed.head()

,depth,table,x,y,z,cut,color,clarity,carat
35965,0.608333,0.288462,0.367784,0.067402,0.080818,1.0,5.0,5.0,0.25
52281,0.522222,0.250000,0.562384,0.103056,0.117610,4.0,0.0,2.0,0.84
6957,0.502778,0.288462,0.610801,0.110526,0.125472,3.0,0.0,3.0,1.05
9163,0.491667,0.250000,0.608007,0.110357,0.124214,4.0,4.0,1.0,1.02
50598,0.522222,0.269231,0.505587,0.092869,0.105975,4.0,4.0,4.0,0.61


In [10]:
X_test_transformed = ct.transform(X_test)
X_test_transformed.head()

,depth,table,x,y,z,cut,color,clarity,carat
1388,0.530556,0.250000,0.369646,0.067912,0.077673,4.0,3.0,6.0,0.24
50052,0.472222,0.269231,0.506518,0.092020,0.102516,2.0,4.0,5.0,0.58
41645,0.530556,0.230769,0.443203,0.080475,0.092767,4.0,5.0,5.0,0.40
42377,0.494444,0.269231,0.458101,0.083022,0.093711,3.0,5.0,5.0,0.43
17244,0.536111,0.230769,0.692737,0.125127,0.144969,4.0,5.0,1.0,1.55


In [73]:
query = X_test_transformed.sample(1)

In [74]:
from sklearn.metrics.pairwise import euclidean_distances

In [75]:
distances = euclidean_distances(X_train_transformed,query)
distances.flatten()

array([3.16707629, 5.41272481, 5.25122512, ..., 3.74178637, 5.51104796,
       2.59309041])

In [76]:
sorted_sim = np.argsort(distances.flatten())
sorted_sim[:5]

array([30790, 34068, 33665,  7870,  4208], dtype=int64)

In [77]:
y[sorted_sim[:5]].values

array([ 742,  851,  835, 4309, 3565], dtype=int64)

In [78]:
pd.DataFrame(y[sorted_sim[:5]]).mean()

price    2060.4
dtype: float64

In [79]:
from sklearn.neighbors import KNeighborsRegressor

In [80]:
knn = KNeighborsRegressor(n_neighbors=5,p=2,metric='minkowski')

In [81]:
knn.fit(X_train_transformed,y_train)

KNeighborsRegressor()

In [82]:
y_pred = knn.predict(query)
y_pred

array([777.4])